### Import dependencies

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime as dt
from sklearn.metrics import confusion_matrix, classification_report

### Merge datasets

In [2]:
# Read in data
cpi = pd.read_csv("resources/cpi_final.csv")
gdp = pd.read_csv("resources/gdp_final.csv")
gdp_pct = pd.read_csv("resources/gdp_pct_chg_final.csv")
houst = pd.read_csv("resources/housing_starts_final.csv")
opg = pd.read_csv("resources/output_gap_final.csv")
rec_dt = pd.read_csv("resources/recession_dates_final.csv")
unrate = pd.read_csv("resources/unemployment_rate_final.csv")
fed_funds = pd.read_csv("resources/fed_funds_final.csv")
yield10_2 = pd.read_csv("resources/10YT_minus_2YT_final.csv")
fred = pd.read_csv("resources/FRED_data.csv")

In [3]:
# Combine all data sets into one data frame
dfs = [cpi, gdp, gdp_pct, houst, opg, rec_dt, unrate, fed_funds, yield10_2, fred]
df = reduce(lambda left,right: pd.merge(left,right,on=['quarter'],how='outer'), dfs)

In [4]:
# Sort data frame by quarter
df = df.sort_values(by=['quarter'])

In [5]:
# Drop date columns
df = df.drop(columns=['date_x','date_y'])

# Rename target column
df = df.rename(columns={'target':'recession_actual'})

In [6]:
# Set index to quarter
df = df.set_index('quarter')

In [7]:
# Save 2019 Q1 & Q2
df_2019 = df.iloc[[-4,-3],:]
df_2019 = df_2019.drop(columns=['recession_actual'])
df_2019

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,,
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,4.133333,2.401311,0.083088,0.004646,0.170000,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250
2019Q2,255.139333,21340.267,4.7,1255.666667,0.828815,3.500000,2.397813,-0.001457,0.024002,0.213333,0.254902,2.4,1.6,1.811376,103.20060,3749.471,1.457,322500.0,4.572,14511.176


In [8]:
# Drop rows with missing values
df = df.dropna()
df.tail()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,recession_actual,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,,,
2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,0.0,4.333333,1.448966,0.204683,0.083902,...,-0.113861,6.9,1.8,2.214194,104.59493,3542.412,1.451,331800.0,4.597,13728.357
2018Q2,250.578667,20510.177,7.1,1259.666667,0.589182,0.0,3.833333,1.727176,0.192007,0.075492,...,-0.251397,2.7,2.0,2.711887,103.33928,3561.592,1.461,315600.0,4.592,13939.828
2018Q3,251.828667,20749.752,4.8,1233.000000,0.821959,0.0,3.866667,1.923492,0.113663,0.047184,...,-0.432836,3.3,2.0,2.640940,103.69309,3683.981,1.462,330900.0,4.587,14114.559
2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,0.066218,...,-0.078947,2.8,1.9,2.203131,105.15026,3725.234,1.462,322800.0,4.582,14211.920
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,0.004646,...,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250


### Shift data with sliding window technique

In [9]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [10]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [11]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()

In [12]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [13]:
# Drop target
df_q1 = df_q1.drop(columns=['recession_1q_out'])
df_q2 = df_q2.drop(columns=['recession_2q_out'])
df_q4 = df_q4.drop(columns=['recession_4q_out'])

In [14]:
# Define X
X_q1 = df_q1
X_q2 = df_q2
X_q4 = df_q4

### Split and scale data

In [15]:
# Split data into training and testing
X1_train, X1_test, y1_train, y1_test=train_test_split(X_q1, y1, train_size=0.8, random_state=42, stratify=y1)
X2_train, X2_test, y2_train, y2_test=train_test_split(X_q2, y2, train_size=0.8, random_state=42, stratify=y2)
X3_train, X3_test, y3_train, y3_test=train_test_split(X_q4, y3, train_size=0.8, random_state=42, stratify=y3)

In [16]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

# X full scaler object
X1_full_scaler = StandardScaler().fit(X_q1)
X2_full_scaler = StandardScaler().fit(X_q2)
X3_full_scaler = StandardScaler().fit(X_q4)

In [17]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

# Scale full X data (no splits)
X1_full_scaled = X1_full_scaler.transform(X_q1)
X2_full_scaled = X2_full_scaler.transform(X_q2)
X3_full_scaled = X3_full_scaler.transform(X_q4)

### Reshape data to fit LSTM format

In [18]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [19]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [20]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

In [21]:
# Reshape X_full
reshaped_X1_full = reshape_data(X1_full_scaled)
reshaped_X2_full = reshape_data(X2_full_scaled)
reshaped_X3_full = reshape_data(X3_full_scaled)

# Build Model

In [22]:
# Initialize model
model = Sequential()

In [23]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

## Train and predict on X1-Y1 data (recession 1 quarter out)

In [25]:
# Fit the model to the training data
model.fit(reshaped_X1_train_scaled, y1_train, validation_split=0.2, epochs=100, shuffle=True, verbose=2)

Train on 108 samples, validate on 28 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
108/108 - 19s - loss: 1.2016 - acc: 0.5093 - val_loss: 0.6860 - val_acc: 0.6429
Epoch 2/100
108/108 - 2s - loss: 1.0185 - acc: 0.5093 - val_loss: 0.6783 - val_acc: 0.6429
Epoch 3/100
108/108 - 2s - loss: 0.5950 - acc: 0.6944 - val_loss: 0.6722 - val_acc: 0.6429
Epoch 4/100
108/108 - 2s - loss: 0.5807 - acc: 0.7407 - val_loss: 0.6706 - val_acc: 0.6429
Epoch 5/100
108/108 - 2s - loss: 0.5296 - acc: 0.7685 - val_loss: 0.6693 - val_acc: 0.6429
Epoch 6/100
108/108 - 2s - loss: 0.6572 - acc: 0.7593 - val_loss: 0.6673 - val_acc: 0.6429
Epoch 7/100
108/108 - 2s - loss: 0.4575 - acc: 0.8148 - val_loss: 0.6695 - val_acc: 0.6429
Epoch 8/100
108/108 - 2s - loss: 0.5285 - acc: 0.8056 - val_loss: 0.6764 - val_acc: 0.6429
Epoch 9/100
108/108 - 2s - loss: 0.3439 - acc: 0.8611 - val_loss: 0.6808 - val_acc: 0.6429
Epoch 10/100
108/108 - 1s - loss: 0.4909 

108/108 - 2s - loss: 0.1928 - acc: 0.9259 - val_loss: 1.9647 - val_acc: 0.6429
Epoch 87/100
108/108 - 2s - loss: 0.1348 - acc: 0.9444 - val_loss: 1.9396 - val_acc: 0.6429
Epoch 88/100
108/108 - 2s - loss: 0.1894 - acc: 0.9167 - val_loss: 1.9081 - val_acc: 0.6429
Epoch 89/100
108/108 - 2s - loss: 0.1622 - acc: 0.9352 - val_loss: 1.8279 - val_acc: 0.6429
Epoch 90/100
108/108 - 2s - loss: 0.1454 - acc: 0.9444 - val_loss: 1.6671 - val_acc: 0.6429
Epoch 91/100
108/108 - 2s - loss: 0.1595 - acc: 0.9352 - val_loss: 1.5894 - val_acc: 0.6429
Epoch 92/100
108/108 - 2s - loss: 0.2095 - acc: 0.9074 - val_loss: 1.6930 - val_acc: 0.6429
Epoch 93/100
108/108 - 2s - loss: 0.2240 - acc: 0.9074 - val_loss: 1.7732 - val_acc: 0.6786
Epoch 94/100
108/108 - 2s - loss: 0.1821 - acc: 0.9352 - val_loss: 1.9055 - val_acc: 0.6786
Epoch 95/100
108/108 - 2s - loss: 0.1595 - acc: 0.9167 - val_loss: 2.0367 - val_acc: 0.6429
Epoch 96/100
108/108 - 2s - loss: 0.1617 - acc: 0.9259 - val_loss: 2.1202 - val_acc: 0.6786
E

In [26]:
# Evaluate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

34/34 - 0s - loss: 1.0522 - acc: 0.8529


In [27]:
# Make predictions using test data
predictions1_prob = model.predict_proba(reshaped_X1_test_scaled)
predictions1_class = model.predict_classes(reshaped_X1_test_scaled)

In [28]:
# Compare results
one_qtr_out = pd.DataFrame({"Predicted Probability of No Recession":predictions1_prob[:,0], "Predicted Probability of a Recession":predictions1_prob[:,1], "Actual":y1_test})
one_qtr_out.head()

,Predicted Probability of No Recession,Predicted Probability of a Recession,Actual
quarter,,,
2006Q1,0.841603,0.158397,0.0
1988Q2,0.995699,0.004301,0.0
2012Q1,0.838005,0.161995,0.0
2015Q1,0.993302,0.006698,0.0
2009Q3,0.600095,0.399905,0.0


#### Confusion Matrix on X1-Y1 data (recession 1 quarter out)

In [29]:
# Create confusion matrix on X1 model
con_mat = confusion_matrix(y1_test, predictions1_class)
print(con_mat)

[[29  0]
 [ 5  0]]


In [30]:
# Score model
print(classification_report(y1_test, predictions1_class))

              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92        29
         1.0       0.00      0.00      0.00         5

    accuracy                           0.85        34
   macro avg       0.43      0.50      0.46        34
weighted avg       0.73      0.85      0.79        34



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
# Save model
name1 = f"shuffled-1q-out-{dt.datetime.now()}"
model.save(f"models/{name1}.h5")

### Predict on 2019

In [32]:
# Scale 2019 data 
scaled_X1_2019 = X1_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X1_2019 = reshape_data(scaled_X1_2019)

# Predict on 2019
pred_X1_2019 = model.predict_proba(reshaped_X1_2019)
print(f"2019Q1 No Recession Probability: {pred_X1_2019[0][0]*100}")
print(f"2019Q1 Recession Probability: {pred_X1_2019[0][1]*100}")
print(f"2019Q2 No Recession Probability: {pred_X1_2019[1][0]*100}")
print(f"2019Q2 Recession Probability: {pred_X1_2019[1][1]*100}")

2019Q1 No Recession Probability: 89.24188613891602
2019Q1 Recession Probability: 10.758111625909805
2019Q2 No Recession Probability: 99.81295466423035
2019Q2 Recession Probability: 0.18704284448176622


#### Predict on full X1

In [33]:
pred_X1_full = model.predict_classes(reshaped_X1_full)

# Preview results
X1_full_results = pd.DataFrame({"Predicted":pred_X1_full, "Actual":y1})
# X1_full_results.loc[X1_full_results["Actual"]==1]

# Export results for graphing
X1_full_results.to_csv(f"resources/predictions/X1_S_VS20_{dt.datetime.now()}.csv")

### Train and predict on X2-Y2 data (recession 2 quarters out)

In [34]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, validation_split=0.2, epochs=100, shuffle=True, verbose=2)

Train on 108 samples, validate on 27 samples
Epoch 1/100
108/108 - 2s - loss: 0.4958 - acc: 0.9167 - val_loss: 1.3198 - val_acc: 0.7037
Epoch 2/100
108/108 - 2s - loss: 0.3351 - acc: 0.9074 - val_loss: 1.3271 - val_acc: 0.6667
Epoch 3/100
108/108 - 2s - loss: 0.2582 - acc: 0.9167 - val_loss: 1.3935 - val_acc: 0.6296
Epoch 4/100
108/108 - 2s - loss: 0.3207 - acc: 0.8889 - val_loss: 1.3654 - val_acc: 0.6296
Epoch 5/100
108/108 - 1s - loss: 0.2283 - acc: 0.9074 - val_loss: 1.2837 - val_acc: 0.6296
Epoch 6/100
108/108 - 1s - loss: 0.2125 - acc: 0.8889 - val_loss: 1.2453 - val_acc: 0.6667
Epoch 7/100
108/108 - 1s - loss: 0.1774 - acc: 0.9259 - val_loss: 1.2160 - val_acc: 0.6667
Epoch 8/100
108/108 - 1s - loss: 0.2213 - acc: 0.8981 - val_loss: 1.2086 - val_acc: 0.6667
Epoch 9/100
108/108 - 2s - loss: 0.2393 - acc: 0.9074 - val_loss: 1.2797 - val_acc: 0.6296
Epoch 10/100
108/108 - 2s - loss: 0.2023 - acc: 0.8981 - val_loss: 1.3894 - val_acc: 0.5926
Epoch 11/100
108/108 - 2s - loss: 0.2079 - a

Epoch 90/100
108/108 - 1s - loss: 0.0457 - acc: 0.9907 - val_loss: 2.7490 - val_acc: 0.6296
Epoch 91/100
108/108 - 1s - loss: 0.0869 - acc: 0.9630 - val_loss: 2.7109 - val_acc: 0.6296
Epoch 92/100
108/108 - 1s - loss: 0.0469 - acc: 0.9907 - val_loss: 2.6720 - val_acc: 0.6296
Epoch 93/100
108/108 - 1s - loss: 0.0235 - acc: 1.0000 - val_loss: 2.6671 - val_acc: 0.6296
Epoch 94/100
108/108 - 2s - loss: 0.0256 - acc: 1.0000 - val_loss: 2.6501 - val_acc: 0.6296
Epoch 95/100
108/108 - 2s - loss: 0.0212 - acc: 0.9907 - val_loss: 2.6238 - val_acc: 0.6296
Epoch 96/100
108/108 - 2s - loss: 0.0301 - acc: 0.9907 - val_loss: 2.5418 - val_acc: 0.6667
Epoch 97/100
108/108 - 2s - loss: 0.0207 - acc: 1.0000 - val_loss: 2.4993 - val_acc: 0.6667
Epoch 98/100
108/108 - 2s - loss: 0.0422 - acc: 0.9815 - val_loss: 2.4833 - val_acc: 0.6667
Epoch 99/100
108/108 - 2s - loss: 0.0137 - acc: 1.0000 - val_loss: 2.4707 - val_acc: 0.6667
Epoch 100/100
108/108 - 2s - loss: 0.0450 - acc: 0.9907 - val_loss: 2.4994 - val

In [35]:
# Evaluate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

34/34 - 0s - loss: 0.8645 - acc: 0.8235


In [36]:
# Make predictions using test data
predictions2_prob = model.predict_proba(reshaped_X2_test_scaled)
predictions2_class = model.predict_classes(reshaped_X2_test_scaled)

In [37]:
# Compare results
two_qtrs_out = pd.DataFrame({"Predicted Probability of No Recession":predictions2_prob[:,0], "Predicted Probability of a Recession":predictions2_prob[:,1], "Actual":y2_test})
two_qtrs_out.head()

,Predicted Probability of No Recession,Predicted Probability of a Recession,Actual
quarter,,,
2006Q1,0.999968,0.000032,0.0
2015Q1,0.999999,0.000001,0.0
2012Q1,0.999975,0.000025,0.0
1996Q2,0.999995,0.000005,0.0
2009Q3,0.052108,0.947892,0.0


#### Confusion Matrix on X2-Y2 data (recession 2 quarters out)

In [38]:
# Create confusion matrix on X2 model
con_mat = confusion_matrix(y2_test, predictions2_class)
print(con_mat)

[[27  2]
 [ 4  1]]


In [39]:
# Score model
print(classification_report(y2_test, predictions2_class))

              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90        29
         1.0       0.33      0.20      0.25         5

    accuracy                           0.82        34
   macro avg       0.60      0.57      0.57        34
weighted avg       0.79      0.82      0.80        34



In [40]:
# Save model
name2 = f"shuffled-2q-out-{dt.datetime.now()}"
model.save(f"models/{name2}.h5")

### Predict on 2019

In [41]:
# Scale 2019 data 
scaled_X2_2019 = X2_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X2_2019 = reshape_data(scaled_X2_2019)

# Predict on 2019
pred_X2_2019 = model.predict_proba(reshaped_X2_2019)
print(f"2019Q1 No Recession Probability: {pred_X2_2019[0][0]*100}")
print(f"2019Q1 Recession Probability: {pred_X2_2019[0][1]*100}")
print(f"2019Q2 No Recession Probability: {pred_X2_2019[1][0]*100}")
print(f"2019Q2 Recession Probability: {pred_X2_2019[1][1]*100}")
print(pred_X2_2019)

2019Q1 No Recession Probability: 99.99921321868896
2019Q1 Recession Probability: 0.0007919996278360486
2019Q2 No Recession Probability: 99.9991774559021
2019Q2 Recession Probability: 0.0008209300176531542
[[9.9999213e-01 7.9199963e-06]
 [9.9999177e-01 8.2093002e-06]]


#### Predict on full X2

In [42]:
pred_X2_full = model.predict_classes(reshaped_X2_full)
X2_full_results = pd.DataFrame({"Predicted":pred_X2_full, "Actual":y2})
X2_full_results.to_csv(f"resources/predictions/X2_S_VS20_{dt.datetime.now()}.csv")
# X2_full_results.loc[X2_full_results["Actual"]==1]

### Train and predict on X3-Y3 data (recession 4 quarters out)

In [43]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, validation_split=0.2, epochs=100, shuffle=True, verbose=2)

Train on 106 samples, validate on 27 samples
Epoch 1/100
106/106 - 2s - loss: 0.8186 - acc: 0.8774 - val_loss: 1.8092 - val_acc: 0.5926
Epoch 2/100
106/106 - 2s - loss: 0.4222 - acc: 0.8868 - val_loss: 1.9568 - val_acc: 0.5556
Epoch 3/100
106/106 - 2s - loss: 0.4096 - acc: 0.8491 - val_loss: 2.0218 - val_acc: 0.4815
Epoch 4/100
106/106 - 2s - loss: 0.3210 - acc: 0.8396 - val_loss: 1.6848 - val_acc: 0.4074
Epoch 5/100
106/106 - 2s - loss: 0.2690 - acc: 0.8585 - val_loss: 1.3635 - val_acc: 0.4444
Epoch 6/100
106/106 - 2s - loss: 0.2795 - acc: 0.8774 - val_loss: 1.1538 - val_acc: 0.4444
Epoch 7/100
106/106 - 2s - loss: 0.2638 - acc: 0.8774 - val_loss: 1.0957 - val_acc: 0.4444
Epoch 8/100
106/106 - 2s - loss: 0.2974 - acc: 0.8868 - val_loss: 1.1147 - val_acc: 0.4815
Epoch 9/100
106/106 - 2s - loss: 0.2082 - acc: 0.9245 - val_loss: 1.1104 - val_acc: 0.4074
Epoch 10/100
106/106 - 2s - loss: 0.1967 - acc: 0.9057 - val_loss: 1.0775 - val_acc: 0.5185
Epoch 11/100
106/106 - 2s - loss: 0.1637 - a

Epoch 90/100
106/106 - 2s - loss: 0.0596 - acc: 0.9811 - val_loss: 1.7467 - val_acc: 0.7407
Epoch 91/100
106/106 - 2s - loss: 0.0863 - acc: 0.9434 - val_loss: 1.8213 - val_acc: 0.7037
Epoch 92/100
106/106 - 2s - loss: 0.0739 - acc: 0.9623 - val_loss: 1.8793 - val_acc: 0.7407
Epoch 93/100
106/106 - 1s - loss: 0.0733 - acc: 0.9528 - val_loss: 1.9053 - val_acc: 0.7407
Epoch 94/100
106/106 - 1s - loss: 0.0524 - acc: 0.9811 - val_loss: 1.8995 - val_acc: 0.7407
Epoch 95/100
106/106 - 2s - loss: 0.0785 - acc: 0.9528 - val_loss: 1.8211 - val_acc: 0.7407
Epoch 96/100
106/106 - 1s - loss: 0.0952 - acc: 0.9528 - val_loss: 1.7832 - val_acc: 0.7778
Epoch 97/100
106/106 - 2s - loss: 0.1706 - acc: 0.9434 - val_loss: 1.7792 - val_acc: 0.7778
Epoch 98/100
106/106 - 2s - loss: 0.0972 - acc: 0.9623 - val_loss: 1.8807 - val_acc: 0.7407
Epoch 99/100
106/106 - 1s - loss: 0.1175 - acc: 0.9434 - val_loss: 1.8280 - val_acc: 0.7778
Epoch 100/100
106/106 - 2s - loss: 0.0912 - acc: 0.9434 - val_loss: 1.8135 - val

In [44]:
# Validate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

34/34 - 0s - loss: 0.6712 - acc: 0.8824


In [45]:
# Make predictions using test data
predictions3_prob = model.predict_proba(reshaped_X3_test_scaled)
predictions3_class = model.predict_classes(reshaped_X3_test_scaled)

In [46]:
# Compare results
four_qtrs_out = pd.DataFrame({"Predicted Probability of No Recession":predictions3_prob[:,0], "Predicted Probability of a Recession":predictions3_prob[:,1], "Actual":y3_test})
four_qtrs_out.head()

,Predicted Probability of No Recession,Predicted Probability of a Recession,Actual
quarter,,,
2007Q3,0.984044,1.595584e-02,1.0
1976Q4,1.000000,1.163328e-07,0.0
2018Q1,0.999986,1.399406e-05,0.0
2009Q3,0.992403,7.597438e-03,0.0
1996Q2,0.999996,4.433415e-06,0.0


#### Confusion Matrix on X3-Y3 data (recession 4 quarters out)

In [47]:
# Create confusion matrix on X3 model
con_mat = confusion_matrix(y3_test, predictions3_class)
print(con_mat)

[[28  1]
 [ 3  2]]


In [48]:
# Score model
print(classification_report(y3_test, predictions3_class))

              precision    recall  f1-score   support

         0.0       0.90      0.97      0.93        29
         1.0       0.67      0.40      0.50         5

    accuracy                           0.88        34
   macro avg       0.78      0.68      0.72        34
weighted avg       0.87      0.88      0.87        34



In [49]:
# Save model
name3 = f"shuffled-4q-out-{dt.datetime.now()}"
model.save(f"models/{name3}.h5")

### Predict on 2019

In [50]:
# Scale 2019 data 
scaled_X3_2019 = X3_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X3_2019 = reshape_data(scaled_X3_2019)

# Predict on 2019
pred_X3_2019 = model.predict_proba(reshaped_X3_2019)
print(f"2019Q1 No Recession Probability: {pred_X3_2019[0][0]*100}")
print(f"2019Q1 Recession Probability: {pred_X3_2019[0][1]*100}")
print(f"2019Q2 No Recession Probability: {pred_X3_2019[1][0]*100}")
print(f"2019Q2 Recession Probability: {pred_X3_2019[1][1]*100}")
print(pred_X3_2019)

2019Q1 No Recession Probability: 99.99922513961792
2019Q1 Recession Probability: 0.0007724521310592536
2019Q2 No Recession Probability: 99.99970197677612
2019Q2 Recession Probability: 0.00029544448807428125
[[9.9999225e-01 7.7245213e-06]
 [9.9999702e-01 2.9544449e-06]]


#### Predict on full X3

In [51]:
pred_X3_full = model.predict_classes(reshaped_X3_full)

# Preview results
X3_full_results = pd.DataFrame({"Predicted":pred_X3_full, "Actual":y3})
# X3_full_results.loc[X3_full_results["Actual"]==1]

# Export results for graphing
X3_full_results.to_csv(f"resources/predictions/X3_S_VS20_{dt.datetime.now()}.csv")